In [ ]:
#
# Tests the ability to use springs layout selectively on subgraphs
# ... so, if you keep the placement of the nodes from an initial all-nodes layout
# ... then you can update the placement of a subset of the nodes
#
# However... because that's not the only transform done to make this all work
# (think about the treemap component placement), then there's always a scaling factor
# involved... and that breaks the ability to apply the spring layout to a subset of 
# nodes...
#

import polars   as pl
import networkx as nx
import random
import copy
import rtsvg
import linknode_graph_patterns
from rtsvg.polars_spring_layout import PolarsSpringLayout
rt   = rtsvg.RACETrack()
'''
g    = linknode_graph_patterns.LinkNodeGraphPatterns().__pattern_mesh__()
_lu_ = {'fm':[], 'to':[]}
for n in g.nodes():
    for nbor in g.neighbors(n): _lu_['fm'].append(n), _lu_['to'].append(nbor)
df        = pl.DataFrame(_lu_)
_to_fix_  = {'node_0_5', 'node_7_4', 'node_8_6', 'node_5_8'}
'''

_base_network_ = '0'
_base_dir_     = '../../data/stanford/facebook/'
_layout_file_  = _base_dir_ + _base_network_ + '.layout.parquet'
_edges_ = open(_base_dir_ + _base_network_ + '.edges', 'rt').read()
_lu_ = {'fm':[], 'to':[]}
for _edge_ in _edges_.split('\n'):
    if _edge_ == '': continue
    _lu_['fm'].append(_edge_.split(' ')[0])
    _lu_['to'].append(_edge_.split(' ')[1])
df        = pl.DataFrame(_lu_)
_relates_ = [('fm','to')]
g         = rt.createNetworkXGraph(df, _relates_)
_to_fix_  = {'110', '193', '201', '245', '259', '264', '61', '8', '91'}

_relates_ = [('fm','to')]
_colors_  = {n:'red' for n in _to_fix_}
_psl_     = PolarsSpringLayout(g)
pos       = _psl_.results()
new_pos   = copy.deepcopy(pos)
for n in _to_fix_: new_pos[n] = (new_pos[n][0] + 100*(random.random()-0.5), new_pos[n][1] + 100*(random.random()-0.5))
new_new_pos = copy.deepcopy(new_pos)
new_new_pos = PolarsSpringLayout(g, pos=new_new_pos, static_nodes=set(g.nodes()) - _to_fix_, normalize_coordinates=True).results()
params      = {'w':384, 'h':384, 'node_color':_colors_}
rt.tile([rt.link(df, _relates_, pos,         **params),
         rt.link(df, _relates_, new_pos,     **params),
         rt.link(df, _relates_, new_new_pos, **params)], spacer=10)

In [ ]:
#
# So... maybe it's possible to rescale the graph based on the expected distances
# ... in this case (mesh graph), all edges should be 1.0 ... so... how to take
# ... a random set of positions & then rescale them?
#
_d_sum_, _d_samples_, _rms_sum_ = 0.0, 0, 0.0
for n in g.nodes():
    for nbor in g.neighbors(n):
        _w_                  =  g[n][nbor]['weight']
        _xy_n_, _xy_nbor_    =  pos[n], pos[nbor]
        _d_                  =  rt.segmentLength((_xy_n_, _xy_nbor_))
        _w_diff_             =  _w_ - _d_
        _rms_sum_            += _w_diff_**2
        _d_sum_, _d_samples_ = _d_sum_ + _d_, _d_samples_ + 1
print(_d_sum_, _d_samples_, _d_sum_ / _d_samples_) # 359.24853900508407 288 1.2473907604343197 -- example from the mesh graph

In [ ]:
#
# Grid Search Version of This Optimization
#
_lu_ = {'x_scale':[], 'y_scale':[], 'rms':[]}
_inc_     = 0.05
_x_scale_ = 0.5
while _x_scale_ < 1.5:
    _y_scale_ = 0.5
    while _y_scale_ < 1.5:
        _rms_ = _psl_.rootMeanSquareError(_x_scale_, _y_scale_)
        _lu_['x_scale'].append(_x_scale_)
        _lu_['y_scale'].append(_y_scale_)
        _lu_['rms'].append(_rms_)
        _y_scale_ += _inc_
    _x_scale_ += _inc_

rt.tile([rt.xy(pl.DataFrame(_lu_), x_field='x_scale', y_field='rms', line_groupby_field='y_scale', color_by='y_scale', dot_size=None),
         rt.xy(pl.DataFrame(_lu_), x_field='y_scale', y_field='rms', line_groupby_field='x_scale', color_by='x_scale', dot_size=None)], spacer=10)

In [ ]:
#
# From Google AI
#
import numpy as np

def calculate_parabola_coefficients(p1, p2, p3):
    """
    Calculates the coefficients (a, b, c) of a parabola y = ax^2 + bx + c
    that passes through three given points.

    Args:
        p1 (tuple): The first point (x1, y1).
        p2 (tuple): The second point (x2, y2).
        p3 (tuple): The third point (x3, y3).

    Returns:
        tuple: A tuple containing the coefficients (a, b, c).
               Returns None if the points are collinear or cannot form a unique parabola.
    """
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3

    # Construct the matrix A and vector B for the system of equations
    # A * [a, b, c]^T = B
    A = np.array([
        [x1**2, x1, 1],
        [x2**2, x2, 1],
        [x3**2, x3, 1]
    ])

    B = np.array([y1, y2, y3])

    try:
        # Solve the system of equations for a, b, c
        coefficients = np.linalg.solve(A, B)
        return tuple(coefficients)
    except np.linalg.LinAlgError:
        # This error occurs if the matrix A is singular,
        # which happens when the three points are collinear.
        print("Error: The three points are collinear or cannot form a unique parabola.")
        return None

# Example usage:
point1 = (1, 2)
point2 = (2, 1)
point3 = (3, 4)

a, b, c = calculate_parabola_coefficients(point1, point2, point3)

if a is not None: print(f"Parabola coefficients: a={a}, b={b}, c={c}")

In [ ]:
_lu_ = {'x_scale':[], 'rms':[], 'group':[]}
x_scale = 0.1
while x_scale < 2.0:
    rms = _psl_.rootMeanSquareError(x_scale, 1.0)
    _lu_['x_scale'].append(x_scale), _lu_['rms'].append(rms), _lu_['group'].append('rms')
    x_scale += 0.1

for i in range(1,len(_lu_['rms'])-1):
    if _lu_['rms'][i] < _lu_['rms'][i-1] and _lu_['rms'][i] < _lu_['rms'][i+1]:
        print(f'Found! ({i=})')
        p0 = (_lu_['x_scale'][i-1], _lu_['rms'][i-1])
        p1 = (_lu_['x_scale'][i],   _lu_['rms'][i])
        p2 = (_lu_['x_scale'][i+1], _lu_['rms'][i+1])

_a_, _b_, _c_ = calculate_parabola_coefficients(p0, p1, p2) 
_a_, _b_, _c_ = float(a), float(b), float(c)

if _a_ > 0.0: # opens upward...
    x = -_b_ / 2 * _a_
    y = _a_ * x**2 + _b_ * x + _c_ 
else: raise Exception('Parabola opens downward...')
x,y

_lu2_ = {'x_scale':[], 'rms':[], 'group':[]}
x_scale = 0.1
while x_scale < 2.0:
    _lu2_['x_scale'].append(x_scale), _lu2_['rms'].append(_a_ * x_scale**2 + _b_ * x_scale + _c_), _lu2_['group'].append('formula')
    x_scale += 0.1
_df_ = pl.concat([pl.DataFrame(_lu_), pl.DataFrame(_lu2_)])
rt.tile([rt.xy(_df_, x_field='x_scale', y_field='rms',     w=768, line_groupby_field="group")])